In [1]:
N = ord("O") % 5 + 1
N

5

In [2]:
import cv2
import numpy as np
import pandas as pd
import os

excel_file = 'kr1.xlsx'
df = pd.read_excel(excel_file)
N = 5

data_variant = df.loc[df['N'] == N]

def process_image_from_dataframe(data_row, output_path):

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    color_dict = {
        'red': (255, 0, 0),
        'blue': (0, 0, 255),
        'cian': (0, 255, 255),
        'magenta': (255, 0, 255),
        'yellow': (0, 255, 255)
    }
    color_name = data_row['glasses color']
    color = color_dict.get(color_name, (0, 0, 0))
    thickness = data_row['line width']

    image_path = os.path.join('Images', data_row['file name'])

    print(f'Loading image from: {image_path}')
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Изображение не найдено: {image_path}")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 2:
            eyes = sorted(eyes, key=lambda e: e[0])
            left_eye = eyes[0]
            right_eye = eyes[1]

            lx, ly = (left_eye[0] + left_eye[2] // 2, left_eye[1] + left_eye[3] // 2)
            rx, ry = (right_eye[0] + right_eye[2] // 2, right_eye[1] + right_eye[3] // 2)

            radius = max(left_eye[2], left_eye[3]) // 2
            cv2.circle(roi_color, (lx, ly), radius, color, thickness)

            radius = max(right_eye[2], right_eye[3]) // 2
            cv2.circle(roi_color, (rx, ry), radius, color, thickness)

            cv2.line(roi_color, (lx + radius, ly), (rx - radius, ry), color, thickness)

    face_center = img[y:y+h, x:x+w]
    reduced_img = cv2.resize(face_center, (500, 500))
    cv2.imwrite(output_path, reduced_img)
    print(f"The image is saved along the way: {output_path}")

output_path = 'result.jpg'
process_image_from_dataframe(data_variant.iloc[0], output_path)

Loading image from: Images\ron_wesley.jpg
The image is saved along the way: result.jpg
